# Restautant orders forecasting

## Problem Defination:
This dataset consisted of 33k orders from two Indian takeaway restaurants in London, UK. The purpose of this notebook is to increase cross-selling when the customer performs by applying makert basket analysis (association rules) where the food could be or recommend when the takeaway is performed:

* By predicting the orders, the raw materials and ingredients can be utilised efficiently
* Preparation of frequent and demanding food orders can be improved to avoid getting exhausted and preparation of less frequent orders can be reduced, thus in overall shortening wastage of food and and its resources
* Both X and Y could be placed on the same shelf, so that buyers of one item would be prompted to buy the other;
* Promotional discounts could be applied to only one of the two items;
* X and Y could be combined into a new product.

In [ ]:
import pandas as pd
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import unidecode
import plotly.express as px

Since we have two data sets, we need to merge the information into one for easier analysis. To differentiate it between restaurant orders, first and second, a specific column was created for this purpose and remove the total_products column because it is a sum of the quantity of items in each order.

In [30]:
orders_first_restaurant = pd.read_csv('./restaurant-1-orders.csv')
orders_first_restaurant['restaurant'] = '1 - restaurant'

orders_second_restaurant = pd.read_csv('./restaurant-2-orders.csv')
orders_second_restaurant['restaurant'] = '2 - restaurant'
orders_second_restaurant.rename(columns={'Order ID':'Order Number'},inplace=True)

orders = pd.concat([orders_first_restaurant,orders_second_restaurant])
orders.drop('Total products',axis=1,inplace=True)

In [31]:
def format_columns(column):
    new_column = ' '.join(column.split())
    new_column = new_column.replace(' ','_')
    return new_column.lower()

orders.columns = [format_columns(c) for c in orders.columns]

After data preprocessing

In [32]:
orders.isna().mean()

order_number     0.0
order_date       0.0
item_name        0.0
quantity         0.0
product_price    0.0
restaurant       0.0
dtype: float64

In [33]:
def stats_summary(data):
    
    sales = data['product_price'].sum()
    quantity = data['quantity'].sum()
    orders_count = len(data['order_number'].unique())
    
    avg_price_order = sales / orders_count
    avg_price_food = sales / quantity
    count_food_orders = quantity / orders_count
    
    return {'avg_price_order': avg_price_order,
            'avg_price_food': avg_price_food,
            'count_food_orders': count_food_orders
           }

def print_summary(data):
    
    stats = stats_summary(data)
    
    for key, value in stats.items():
        print(f'{key}: {value:.3}')

As we can see, the difference of the average order price between two restaurants is \\$ 1.44 and the food price of restaurant 1 is 0.16 cents more than the second restaurant, but the quantity of the food per order is higher in the second restaurant, which increases the average order price. The total shows that the average price is \\$ 43.6 and 10 food per order with \\$ 4.15 per food the average.

In [34]:
print('\n1 - restaurant\n')
print_summary(orders[orders['restaurant']=='1 - restaurant'])
print('\n2 - restaurant\n')
print_summary(orders[orders['restaurant']=='2 - restaurant'])

print('\nTotal\n')
print_summary(orders)


1 - restaurant

avg_price_order: 29.5
avg_price_food: 4.25
count_food_orders: 6.94

2 - restaurant

avg_price_order: 31.0
avg_price_food: 4.09
count_food_orders: 7.57

Total

avg_price_order: 43.6
avg_price_food: 4.15
count_food_orders: 10.5


As we can see, the average frequency item is 509, that is, an item appears on average 509 times. When, we look at the median, we see that it is the outliers that pull the average up. The maximum value is 13093, which represents 56.8% of the orders with this item which is Pilau Rice. The frequency percentage of the item is 2.22% (mean / total unit orders) and the median percentage is 0.49%.

In [35]:
items_frequence = orders.groupby('item_name').size().reset_index(name='quantity')
print(items_frequence.describe())
print(f'\ntotal unique orders: {len(orders["order_number"].unique())}')

           quantity
count    381.000000
mean     509.188976
std     1261.237290
min        1.000000
25%       26.000000
50%      113.000000
75%      408.000000
max    13093.000000

total unique orders: 23041


Removing the extra spaces, symbols or accents if any, because maybe there are duplicate items but the same text with another way of writing.

In [36]:
def text_normalization(text):
    new_text = ' '.join(text.split())
    return unidecode.unidecode(new_text.lower())

orders['item_name'] = orders['item_name'].apply(text_normalization)

We need to transform the dataset into a list of items in order and convert it into a matrix (data set) where the columns are items and the rows are the order. If that item belongs to that order the value 1 is assigned, otherwise 0.

In [37]:
# convert the dataframe to list items in order
item_list = orders.groupby('order_number')['item_name'].unique()

# transform the values of the data set to 1 if that item belongs to that order, otherwise 0
te = TransactionEncoder()
oht_orders = te.fit(item_list).transform(item_list, sparse=True)

To save memory, we represent the transaction data in sparse format.  Because, we have 316 items and 23041 orders.

In [38]:
sparse_df_items = pd.DataFrame.sparse.from_spmatrix(oht_orders, columns=te.columns_)

As a threshold for the minimum frequency of a set of items(the support metric), we used the percentage of the average/unique order frequency, which is 2.22% and max len of set of items equals 10. 

In [39]:
frequent_itemsets = apriori(sparse_df_items, min_support=0.02209,max_len=11, use_colnames=True, verbose=1)

Processing 600 combinations | Sampling itemset size 54


We create a copy of frequent item sets to create a custom output and analyze.

In [40]:
frequent_itemsets_plot = frequent_itemsets.copy()
frequent_itemsets_plot['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets_plot['support'] = (frequent_itemsets_plot['support'] * 100).round(2)
frequent_itemsets_plot["itemsets"] = frequent_itemsets_plot["itemsets"].apply(lambda x: ', '.join(list(x))).astype("str")

We get 556 frequency item sets with the filter we apply. We can see, that most of the item set sizes are 2 and the maximum value is 5. But it is not necessary that this itemset has a high frequency in orders. We have a high std for length equal to 1 indicating item variability and a low variability for length equal to 2 or 3 representing 4.64% and 3.48% respectively on average in the orders.

In [41]:
frequent_itemsets_plot.groupby('length')['support'].describe()

,count,mean,std,min,25%,50%,75%,max
length,,,,,,,,
1,73.0,8.864384,8.976832,2.22,3.2800,5.06,10.580,50.17
2,252.0,4.637302,3.121391,2.21,2.7375,3.51,5.215,24.15
3,197.0,3.484112,1.451654,2.21,2.5400,2.99,3.920,12.07
4,43.0,2.908140,0.703607,2.27,2.4300,2.68,3.100,5.01
5,1.0,2.280000,NaN,2.28,2.2800,2.28,2.280,2.28


As we can see, 50% of orders have rice pilau and 39.78% have simple papadum and other information is 24.15% of orders have simple papadum and rice pilau in the same order and 19.2% have simple papadum, mango chutney. In addition, 2.28% of orders have simple papadum, onion chutney, mango chutney, mint sauce, rice pilau as items.

In [42]:
top_20_frequence = frequent_itemsets_plot.sort_values('support',ascending=False).head(20).sort_values('support')
fig = px.bar(top_20_frequence, x="support", y="itemsets", orientation='h', text='support')
fig.update_traces(textposition="outside")
fig.show()

/Users/kevlarn/opt/anaconda3/lib/python3.9/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/kevlarn/opt/anaconda3/lib/python3.9/site-packages/plotly/io/_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



We filter an lift equal to 1 to get only rules that have a probability of buying the antecedents and consequents in the same order. As we can see, we have a high probability of recomedations if the customer buys an item. For example, we have 174 possibilities of recommendations if the customer buys a plain papadum.

### Association Rules

In [22]:
market_basket_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
market_basket_rules.groupby('antecedents').size().sort_values(ascending=False)

antecedents
(plain papadum)                                           174
(pilau rice)                                              166
(mango chutney)                                           105
(naan)                                                     71
(mint sauce)                                               70
                                                         ... 
(pilau rice, mint sauce, garlic naan)                       1
(pilau rice, garlic naan, mango chutney)                    1
(garlic naan, mint sauce, mango chutney)                    1
(keema naan, pilau rice, plain papadum)                     1
(plain papadum, pilau rice, mint sauce, mango chutney)      1
Length: 285, dtype: int64

<a id='item_recommendation'></a>
### Item Recommendation

To filter the best recommendations we will use the highest confidence value for each antecedent. We ran with the top 20 most frequent items and got some recommendations:

* 4.44% (confidence) of those who buys pilau rice, buys paratha as well;
* If one buys garlic naan, it's likely that one has also bought plain papadum with saag aloo. This way, it is possible to create a bundle with these items and apply a discount;
* If one buys pilau rice with chicken tikka masala it's likely that one has also bought madras. This way, it is possible apply discount in the madras.

In [43]:
best_item_recommendations = market_basket_rules.sort_values(['confidence','lift'],ascending=False).drop_duplicates(subset=['antecedents'])
top_20_frequence_items = frequent_itemsets.sort_values('support',ascending=False).head(20)['itemsets']
best_item_recommendations[best_item_recommendations['antecedents'].isin(top_20_frequence_items)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
303,(mango chutney),(plain papadum),0.227811,0.397813,0.192006,0.842827,2.118654,0.101379,3.831372
341,(mint sauce),(plain papadum),0.166225,0.397813,0.133414,0.802611,2.017560,0.067288,3.050764
135,(chicken tikka masala),(pilau rice),0.231197,0.501714,0.158413,0.685189,1.365695,0.042419,1.582807
418,(plain naan),(pilau rice),0.162840,0.501714,0.103728,0.636994,1.269634,0.022029,1.372664
1283,"(plain papadum, mango chutney)",(pilau rice),0.192006,0.501714,0.120698,0.628617,1.252937,0.024366,1.341702
38,(bombay aloo),(pilau rice),0.243522,0.501714,0.151165,0.620745,1.237248,0.028987,1.313854
409,(peshwari naan),(pilau rice),0.155766,0.501714,0.096480,0.619393,1.234552,0.018330,1.309185
369,(naan),(pilau rice),0.289875,0.501714,0.177640,0.612816,1.221445,0.032206,1.286949
214,(keema naan),(pilau rice),0.160410,0.501714,0.097956,0.610660,1.217147,0.017476,1.279822
420,(plain papadum),(pilau rice),0.397813,0.501714,0.241526,0.607135,1.210121,0.041938,1.268338


<a id='conclusions'></a>
### Conclusions:

* The quantity of the food per order is higher in the second restaurant, which increases the average order price with difference \\$ 1.44;
* The average price is \\$ 43.6 and 10 food per order with \\$ 4.15 per food the average;
* The average frequency item is 509, that is, an item appears on average 509 times which represets 2.22%;
* 50,17% of orders has pilau rice;
* 24.15% of orders have simple papadum and rice pilau in the same order 19.2% have simple papadum, mango chutney;
* 2.28% of orders have simple papadum, onion chutney, mango chutney, mint sauce, rice pilau as items;
* 4.44% (confidence) of those who buys pilau rice, buys paratha as well;
* If one buys plain papadum with pilau rice it's likely that one has also bought garlic naan with naan;
* If one buys garlic naan, it's likely that one has also bought plain papadum with saag aloo. This way, it is possible to create a bundle with these items and apply a discount;
* If one buys pilau rice with chicken tikka masala it's likely that one has also bought madras. This way, it is possible apply discount in the madras.